In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torch.cuda.amp import autocast, GradScaler
import numpy as np

# Constants
BATCH_SIZE = 64  # Reduce batch size to save GPU memory
GRADIENT_ACCUMULATION_STEPS = 4  # Accumulate gradients over multiple steps
NUM_CLASSES = 10
EPOCHS = 15

# Transformations for data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Load the STL-10 dataset
train_dataset = datasets.STL10(root='./data', split='train', transform=transform, download=True)
test_dataset = datasets.STL10(root='./data', split='test', transform=transform, download=True)

# Split data into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Create the ResNet-50 model
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES)  # Change the classifier

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Mixed precision setup
scaler = GradScaler()

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer.zero_grad()
accumulated_loss = 0.0

for epoch in range(EPOCHS):
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Use mixed precision for forward and backward passes
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        accumulated_loss += loss.item()

        if (i + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            accumulated_loss = 0.0

    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {accumulated_loss / len(train_loader)}")

# Validation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_accuracy = correct / total
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Testing loop
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/15, Loss: 0.032213262149265835
Epoch 2/15, Loss: 0.018015133719595653
Epoch 3/15, Loss: 0.014982273890858605
Epoch 4/15, Loss: 0.010098736674066574
Epoch 5/15, Loss: 0.009193491663724657
Epoch 6/15, Loss: 0.00870921379990048
Epoch 7/15, Loss: 0.00990933250813257
Epoch 8/15, Loss: 0.0014667879967462448
Epoch 9/15, Loss: 0.00328055801727469
Epoch 10/15, Loss: 0.010020844993137178
Epoch 11/15, Loss: 0.003943062668281888
Epoch 12/15, Loss: 0.0053805747912043615
Epoch 13/15, Loss: 0.0027922355229892427
Epoch 14/15, Loss: 0.004260684614853253
Epoch 15/15, Loss: 0.005423547493086921
Validation Accuracy: 81.50%
Test Accuracy: 81.91%
